Дан файл с архивом изменений температур с 01 февраля 2014 года по 12 марта 2025
Этот файл был преобразован в таблицу с нужными нам датами: с 1 февраля по 10 мая каждого года
Далее этот файл был преобразован в файл-сводку средних значений для декад каждого года. Данные записаны в таблице "температура-по-декадам.xlsx"

In [190]:
import pandas as pd
import numpy as np
import runpy
from IPython.display import display, Latex, Math, HTML

In [191]:
# runpy.run_path('create-work-tables.py')

Выше мы запустили файл-обработчик таблицы

In [192]:
def matrix_print(matrix):
    latex_str = "\\begin{bmatrix}\n"

    for row in matrix:
        latex_str += " & ".join(f"{x:.4f}" for x in row) + " \\\\ \n"

    latex_str += "\\end{bmatrix}"

    return latex_str

In [193]:
df = pd.read_excel('температура-по-декадам.xlsx', engine='openpyxl')

In [194]:
T_max = df['температура'].max()
T_min = df['температура'].min()
print("Минимальное значение температуры: ", T_min, "\nМаксимальное значение температуры: ", T_max)
T_range_count = 10
T_range_len = round((T_max - T_min)/T_range_count + 1e-4, 4)
print("Дискретезируем этот промежуток на", T_range_count,"диапазонов. Длина одного диапазона ", T_range_len)

Минимальное значение температуры:  -16.13 
Максимальное значение температуры:  20.09
Дискретезируем этот промежуток на 10 диапазонов. Длина одного диапазона  3.6221


создаем матрицу переходов

In [195]:
df.columns

Index(['год', 'месяц', 'декада', 'номер декады', 'температура'], dtype='object')

In [196]:
matrix_temp_year = []

for year in range(2014, 2025):
        filter = (
            (df['год'] == year)
        )
        
        df_filtred = df.loc[filter]
        matrix_temp_year.append(df_filtred['температура'].values)
matrix_temp_year = np.array(matrix_temp_year)
        

In [197]:
display(HTML("<h3>Ниже приведена таблица средних температур, где по вертикали годы от 2014 до 2024, а по горизонтали декады от 1 до 10<h3>"), Latex(matrix_print(matrix_temp_year)))

<IPython.core.display.Latex object>

In [198]:
df = pd.read_excel('температура-февраль-2025.xlsx', engine='openpyxl')
T_feb = df['температура'].values[0]

display(HTML(f"<h3>Среднее значение температуры в первой декаде 2025 года в 12:00  =  {T_feb} градусов<h3>"))

In [199]:
T_range = T_min + T_range_len
index_feb = 0

while T_feb >= T_range:
    T_range += T_range_len
    index_feb += 1
    
pi_1 = np.zeros((T_range_count, 1))
pi_1[index_feb][0] = 1
display(Math("\\text{Ниже представлен вектор }\\pi [1]"), Latex(matrix_print(pi_1)))

<IPython.core.display.Math object>

<IPython.core.display.Latex object>

In [200]:
display(HTML('<h1>Вариант 1<h1>'))

In [201]:
latex_str = "\\begin{bmatrix}\n"

count = 0
for row in np.arange(T_min, T_max, T_range_len):
    count += 1
    latex_str += f"{count}" + "\\text{ промежуток: от }" +  f"{row:.4f}" + "\\text{ до }" + f"{row + T_range_len:.4f}" + " \\\\ \n"

latex_str += "\\end{bmatrix}"
display(Latex(latex_str))

<IPython.core.display.Latex object>

In [202]:
matrix_transition = np.zeros((T_range_count, T_range_count))
dict_transition = {}

for i in range(len(matrix_temp_year)):
    T_range = T_min + T_range_len
    index_feb = 0
    
    while matrix_temp_year[i][0] >= T_range:
        T_range += T_range_len
        index_feb += 1
    
    T_range = T_min + T_range_len
    index_may = 0
    while matrix_temp_year[i][9] >= T_range:
        T_range += T_range_len
        index_may += 1
    
    if index_feb not in dict_transition:
        dict_transition[index_feb] = {'count': 0, 'indexes': []}
    dict_transition[index_feb]['count'] += 1
    dict_transition[index_feb]['indexes'].append(index_may)

for i in range(T_range_count):
    if i in dict_transition:
        for j in dict_transition[i]['indexes']:
            matrix_transition[i][j] += 1/dict_transition[i]['count']
            
display(HTML("<h3>Матрица переходов с февраля в май</h3>"))
display(Latex(matrix_print(matrix_transition)))
            

<IPython.core.display.Latex object>

Теперь можно найти распределение вероятностей температур для мая 2025 года
Находится по формуле \\(\pi[10] = P^T \cdot \pi[1]\\)

In [203]:
pi_10 = np.dot(matrix_transition.T, pi_1)
display(Latex(matrix_print(pi_10)))

<IPython.core.display.Latex object>

In [204]:
latex_str = "\\text{В мае будет температура} \\\\ \n"

count = 0
for row in np.arange(T_min, T_max, T_range_len):
    latex_str += "\\text{в промежутке: от }" +  f"{row:.4f}" + "\\text{ до }" + f"{row + T_range_len:.4f}" + "\\text{ с вероятностью }" + f"{pi_10[count][0]:.4f}" + " \\\\ \n"
    count += 1
display(Math(latex_str))

<IPython.core.display.Math object>

In [205]:
display(HTML('<h1>Вариант 2<h1>'))

In [206]:
matrix_transition = np.zeros((T_range_count, T_range_count))
dict_transition = {}

for trans in range(9):
    for i in range(len(matrix_temp_year)):
        T_range = T_min + T_range_len
        index_cur = 0
        
        while matrix_temp_year[i][trans] >= T_range:
            T_range += T_range_len
            index_cur += 1
        
        T_range = T_min + T_range_len
        index_next = 0
        while matrix_temp_year[i][trans+1] >= T_range:
            T_range += T_range_len
            index_next += 1
        
        if index_cur not in dict_transition:
            dict_transition[index_cur] = {'count': 0, 'indexes': []}
        dict_transition[index_cur]['count'] += 1
        dict_transition[index_cur]['indexes'].append(index_next)

for i in range(T_range_count):
    if i in dict_transition:
        for j in dict_transition[i]['indexes']:
            matrix_transition[i][j] += 1/dict_transition[i]['count']

display(HTML("<h3>Матрица вероятностей переходов из любой температуры в любую</h3>"))                
display(Latex(matrix_print(matrix_transition)))

<IPython.core.display.Latex object>

Теперь можно найти распределение вероятностей температур для мая 2025 года
Находится по формуле \\(\pi[10] = (P^T)^9 \cdot \pi[1]\\)

In [207]:
pi_10 = np.dot(np.linalg.matrix_power(matrix_transition.T, 9), pi_1)
display(Latex(matrix_print(pi_10)))

<IPython.core.display.Latex object>

In [208]:
latex_str = "\\text{В мае будет температура} \\\\ \n"

count = 0
for row in np.arange(T_min, T_max, T_range_len):
    latex_str += "\\text{в промежутке: от }" +  f"{row:.4f}" + "\\text{ до }" + f"{row + T_range_len:.4f}" + "\\text{ с вероятностью }" + f"{pi_10[count][0]:.4f}" + " \\\\ \n"
    count += 1
display(Math(latex_str))

<IPython.core.display.Math object>

In [209]:
display(HTML('<h1>Вариант 3<h1>'))

In [210]:
matrix_transition = []
for i in range(9):  
    matrix_transition.append(np.zeros((T_range_count, T_range_count)))


display(HTML("<h3>Матрица вероятностей переходов из любой температуры в любую</h3>")) 
for trans in range(9):
    dict_transition = {}
    for i in range(len(matrix_temp_year)):
        T_range = T_min + T_range_len
        index_cur = 0
        
        while matrix_temp_year[i][trans] >= T_range:
            T_range += T_range_len
            index_cur += 1
        
        T_range = T_min + T_range_len
        index_next = 0
        while matrix_temp_year[i][trans+1] >= T_range:
            T_range += T_range_len
            index_next += 1
        
        if index_cur not in dict_transition:
            dict_transition[index_cur] = {'count': 0, 'indexes': []}
        dict_transition[index_cur]['count'] += 1
        dict_transition[index_cur]['indexes'].append(index_next)

    for i in range(T_range_count):
        if i in dict_transition:
            for j in dict_transition[i]['indexes']:
                matrix_transition[trans][i][j] += 1/dict_transition[i]['count']
    
    display(HTML(f"<h3>Из {trans} в {trans + 1}</h3>"), Latex(matrix_print(matrix_transition[trans])))              

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Теперь можно найти распределение вероятностей температур для мая 2025 года
Находится по формуле \\(\pi[10] = P^T(9,10) \cdot P^T(8,9) \cdot \ldots \cdot P^T(1,2) \pi[1]\\)

In [211]:
tmp_dot = matrix_transition[8].T
for i in range(8):
    tmp_dot = np.dot(tmp_dot, matrix_transition[7-i].T)
pi_10 = np.dot(tmp_dot, pi_1)
display(Latex(matrix_print(pi_10)))

<IPython.core.display.Latex object>

In [212]:
latex_str = "\\text{В мае будет температура} \\\\ \n"

count = 0
for row in np.arange(T_min, T_max, T_range_len):
    latex_str += "\\text{в промежутке: от }" +  f"{row:.4f}" + "\\text{ до }" + f"{row + T_range_len:.4f}" + "\\text{ с вероятностью }" + f"{pi_10[count][0]:.4f}" + " \\\\ \n"
    count += 1
display(Math(latex_str))

<IPython.core.display.Math object>